<a href="https://www.kaggle.com/code/alexgu7104/hemolysisnotebook?scriptVersionId=123062440" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/aacc-2023-helpwithhemolysis/hemolysis_index_results.csv
/kaggle/input/aacc-2023-helpwithhemolysis/redraw_costs.csv
/kaggle/input/aacc-2023-helpwithhemolysis/train.csv
/kaggle/input/aacc-2023-helpwithhemolysis/test.csv
/kaggle/input/aacc-2023-helpwithhemolysis/sample_solution.csv
/kaggle/input/aacc-2023-helpwithhemolysis/redraw_thresholds.csv


In [2]:
df=pd.read_csv('/kaggle/input/aacc-2023-helpwithhemolysis/hemolysis_index_results.csv')
cost=pd.read_csv("/kaggle/input/aacc-2023-helpwithhemolysis/redraw_costs.csv")

In [3]:
redraw = df.query("HEMOLYSIS_INDEX > 75")
df3 = redraw[['COLLECTOR_ID', 'SPECIMEN_ID', 'HEMOLYSIS_INDEX']].copy()
df3 = df3.groupby('COLLECTOR_ID')['SPECIMEN_ID'].nunique().reset_index()
df3 = df3.rename(columns={'SPECIMEN_ID' : 'REDRAW_COUNT'})
df3

,COLLECTOR_ID,REDRAW_COUNT
0,COLLECTOR_101,1
1,COLLECTOR_1013,2
2,COLLECTOR_1028,3
3,COLLECTOR_1029,1
4,COLLECTOR_1031,1
...,...,...
1234,COLLECTOR_981,1
1235,COLLECTOR_984,3
1236,COLLECTOR_989,2
1237,COLLECTOR_991,1


In [4]:
df=df.merge(cost,how="left",on="ENCOUNTER_TYPE")
df

,ORDER_ID,ORDERABLE,PATIENT_ID,PATIENT_AGE,PATIENT_SEX,PATIENT_RACE,NURSING_UNIT,ENCOUNTER_TYPE,MEDICAL_SERVICE,SPECIMEN_ID,SPECIMEN_TYPE,COLLECTOR_ID,DRAW_SITE,COLLECTION_DAY,COLLECTION_TIME,HEMOLYSIS_INDEX,REDRAW_COST
0,ORDER_1179508,Vancomycin Tr,PATIENT_131440,44,Female,White,UNIT_10,Intensive Care,Medical,SPECIMEN_543979,Blood,COLLECTOR_2732,NaN,6,23:21:00,6.0,162.18
1,ORDER_1181190,Comp Met Plas,PATIENT_58856,75,Male,White,UNIT_49,Inpatient,Bone Marrow Transplant,SPECIMEN_544650,Blood,COLLECTOR_522,NaN,6,23:16:00,8.0,357.15
2,ORDER_1181164,Phos Plas,PATIENT_58856,75,Male,White,UNIT_49,Inpatient,Bone Marrow Transplant,SPECIMEN_544650,Blood,COLLECTOR_522,NaN,6,23:16:00,8.0,357.15
3,ORDER_1181191,Magnesium,PATIENT_58856,75,Male,White,UNIT_49,Inpatient,Bone Marrow Transplant,SPECIMEN_544650,Blood,COLLECTOR_522,NaN,6,23:16:00,8.0,357.15
4,ORDER_1180687,Basic Met Plas,PATIENT_99436,40,Female,White,UNIT_161,Intensive Care,Cardiology,SPECIMEN_544632,Blood,COLLECTOR_201,NaN,6,23:18:00,7.0,162.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101169,ORDER_1253155,Phos Plas,PATIENT_74490,80,Male,White,UNIT_45,Inpatient,Bone Marrow Transplant,SPECIMEN_577685,Blood,COLLECTOR_5636,NaN,27,23:10:00,27.0,357.15
101170,ORDER_1253154,Magnesium,PATIENT_74490,80,Male,White,UNIT_45,Inpatient,Bone Marrow Transplant,SPECIMEN_577685,Blood,COLLECTOR_5636,NaN,27,23:10:00,27.0,357.15
101171,ORDER_1253157,Uric Acid,PATIENT_74490,80,Male,White,UNIT_45,Inpatient,Bone Marrow Transplant,SPECIMEN_577685,Blood,COLLECTOR_5636,NaN,27,23:10:00,27.0,357.15
101172,ORDER_1252360,Comp Met Plas,PATIENT_132340,58,Male,White,UNIT_145,Emergency,Neuro Medicine,SPECIMEN_577345,Blood,COLLECTOR_2490,NaN,27,22:55:00,16.0,600.00


In [5]:
#calculate number of specimen per collector
df2 = df[['COLLECTOR_ID', 'SPECIMEN_ID', 'HEMOLYSIS_INDEX']].copy()
df2 = df2.groupby('COLLECTOR_ID')['SPECIMEN_ID'].nunique().reset_index()
df2 = df2.rename(columns={'SPECIMEN_ID' : 'SPECIMEN_COUNT'})
df2

,COLLECTOR_ID,SPECIMEN_COUNT
0,COLLECTOR_101,7
1,COLLECTOR_1013,35
2,COLLECTOR_1028,23
3,COLLECTOR_1029,17
4,COLLECTOR_1031,30
...,...,...
2223,COLLECTOR_991,6
2224,COLLECTOR_992,43
2225,COLLECTOR_993,23
2226,COLLECTOR_995,2


In [6]:
#%% compute median hemolysis index per collector
def compute_median(X:pd.DataFrame)->float:
    '''Return median hemolsysis index across a collectors samples'''
    median=X.drop_duplicates(subset='SPECIMEN_ID')['HEMOLYSIS_INDEX'].median()
    return(median)

median_hi=df.groupby('COLLECTOR_ID').apply(lambda X: compute_median(X))
median_hi=median_hi.reset_index().rename(columns={0:'median_hi'})
median_hi

,COLLECTOR_ID,median_hi
0,COLLECTOR_101,9.0
1,COLLECTOR_1013,12.0
2,COLLECTOR_1028,8.0
3,COLLECTOR_1029,5.0
4,COLLECTOR_1031,15.5
...,...,...
2223,COLLECTOR_991,16.5
2224,COLLECTOR_992,16.0
2225,COLLECTOR_993,11.0
2226,COLLECTOR_995,7.5


In [7]:
#determine specialized encoutner type for each collector
df4 = df[['COLLECTOR_ID', 'ENCOUNTER_TYPE']].drop_duplicates(subset='COLLECTOR_ID').reset_index(drop=True).copy()
df4

,COLLECTOR_ID,ENCOUNTER_TYPE
0,COLLECTOR_2732,Intensive Care
1,COLLECTOR_522,Inpatient
2,COLLECTOR_201,Intensive Care
3,COLLECTOR_3469,Emergency
4,COLLECTOR_2720,Intensive Care
...,...,...
2224,COLLECTOR_4046,Inpatient
2225,COLLECTOR_3746,Outpatient
2226,COLLECTOR_1858,Emergency
2227,COLLECTOR_4625,Inpatient


In [8]:
#join features with train collector ids in sample solution 
train=pd.read_csv('/kaggle/input/aacc-2023-helpwithhemolysis/train.csv')
train=train.merge(median_hi,how='left',on='COLLECTOR_ID')
train=train.merge(df2,how='left',on='COLLECTOR_ID')
train=train.merge(df3,how='left',on='COLLECTOR_ID')
train=train.merge(df4,how='left',on='COLLECTOR_ID')
train=train.merge(cost,how='left',on='ENCOUNTER_TYPE')
train['REDRAW_COUNT'] = train['REDRAW_COUNT'].fillna(0)
train['REDRAW_PERCENT'] = train['REDRAW_COUNT'] / train['SPECIMEN_COUNT']
train

,SPLIT,COLLECTOR_ID,RELEVANCE,median_hi,SPECIMEN_COUNT,REDRAW_COUNT,ENCOUNTER_TYPE,REDRAW_COST,REDRAW_PERCENT
0,TRAIN,COLLECTOR_13,0.00000,29.0,2.0,0.0,Inpatient,357.15,0.000000
1,TRAIN,COLLECTOR_2184,0.00000,18.0,13.0,2.0,Inpatient,357.15,0.153846
2,TRAIN,COLLECTOR_798,0.00000,13.0,39.0,2.0,Intensive Care,162.18,0.051282
3,TRAIN,COLLECTOR_492,0.32436,21.0,20.0,2.0,Intensive Care,162.18,0.100000
4,TRAIN,COLLECTOR_2658,0.00000,10.5,26.0,0.0,Inpatient,357.15,0.000000
...,...,...,...,...,...,...,...,...,...
1338,TRAIN,COLLECTOR_3798,0.00000,33.0,9.0,1.0,Inpatient,357.15,0.111111
1339,TRAIN,COLLECTOR_3397,0.00000,12.5,46.0,3.0,Inpatient,357.15,0.065217
1340,TRAIN,COLLECTOR_33,0.00000,13.0,43.0,2.0,Inpatient,357.15,0.046512
1341,TRAIN,COLLECTOR_3001,0.00000,13.0,3.0,0.0,Inpatient,357.15,0.000000


In [9]:
#remove redundant columns
train = train.drop(['SPLIT', 'SPECIMEN_COUNT', 'COLLECTOR_ID'], axis = 1)
train

,RELEVANCE,median_hi,REDRAW_COUNT,ENCOUNTER_TYPE,REDRAW_COST,REDRAW_PERCENT
0,0.00000,29.0,0.0,Inpatient,357.15,0.000000
1,0.00000,18.0,2.0,Inpatient,357.15,0.153846
2,0.00000,13.0,2.0,Intensive Care,162.18,0.051282
3,0.32436,21.0,2.0,Intensive Care,162.18,0.100000
4,0.00000,10.5,0.0,Inpatient,357.15,0.000000
...,...,...,...,...,...,...
1338,0.00000,33.0,1.0,Inpatient,357.15,0.111111
1339,0.00000,12.5,3.0,Inpatient,357.15,0.065217
1340,0.00000,13.0,2.0,Inpatient,357.15,0.046512
1341,0.00000,13.0,0.0,Inpatient,357.15,0.000000


In [10]:
#filter for only Emergency and Inpatient collectors
emergency = train.loc[(train['ENCOUNTER_TYPE'] == "Emergency") | (train['ENCOUNTER_TYPE'] == "Inpatient")]
emergency = emergency.drop('ENCOUNTER_TYPE', axis=1)
emergency

,RELEVANCE,median_hi,REDRAW_COUNT,REDRAW_COST,REDRAW_PERCENT
0,0.000000,29.0,0.0,357.15,0.000000
1,0.000000,18.0,2.0,357.15,0.153846
4,0.000000,10.5,0.0,357.15,0.000000
7,0.912717,10.0,1.0,357.15,0.045455
8,0.000000,10.0,0.0,357.15,0.000000
...,...,...,...,...,...
1338,0.000000,33.0,1.0,357.15,0.111111
1339,0.000000,12.5,3.0,357.15,0.065217
1340,0.000000,13.0,2.0,357.15,0.046512
1341,0.000000,13.0,0.0,357.15,0.000000


In [11]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [12]:
X = emergency.drop('RELEVANCE', axis=1)
y = emergency['RELEVANCE']

#Select only nonzero relevance scores to train on
X=X[y>0]
y=y[y>0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    "max_depth": 15,
    "eta": 0.3,
    "subsample": 1.0,
    "colsample_bytree": 1.0,
    "learning_rate": 0.35,
    "objective": "reg:squarederror",
    "eval_metric": "rmse"
}

In [14]:
num_round = 100
bst = xgb.train(params, dtrain, num_round)

In [15]:
preds = bst.predict(dtest)
rmse = mean_squared_error(y_test, preds, squared=False)
print("RMSE:", rmse)

RMSE: 3.329161021152597


In [16]:
#calculate features for test set
test=pd.read_csv('/kaggle/input/aacc-2023-helpwithhemolysis/test.csv')
test=test.merge(median_hi,how='left',on='COLLECTOR_ID')
test=test.merge(df2,how='left',on='COLLECTOR_ID')
test=test.merge(df3,how='left',on='COLLECTOR_ID')
test=test.merge(df4,how='left',on='COLLECTOR_ID')
test=test.merge(cost,how='left',on='ENCOUNTER_TYPE')
test['REDRAW_COUNT'] = test['REDRAW_COUNT'].fillna(0)
test['REDRAW_PERCENT'] = test['REDRAW_COUNT'] / test['SPECIMEN_COUNT']
test

,SPLIT,COLLECTOR_ID,median_hi,SPECIMEN_COUNT,REDRAW_COUNT,ENCOUNTER_TYPE,REDRAW_COST,REDRAW_PERCENT
0,TEST,COLLECTOR_2011,32.0,17.0,4.0,Emergency,600.00,0.235294
1,TEST,COLLECTOR_1550,50.0,9.0,3.0,Inpatient,357.15,0.333333
2,TEST,COLLECTOR_4570,8.0,32.0,2.0,Intensive Care,162.18,0.062500
3,TEST,COLLECTOR_3531,9.0,7.0,0.0,Inpatient,357.15,0.000000
4,TEST,COLLECTOR_5020,31.0,18.0,2.0,Emergency,600.00,0.111111
...,...,...,...,...,...,...,...,...
890,TEST,COLLECTOR_2848,14.0,31.0,1.0,Inpatient,357.15,0.032258
891,TEST,COLLECTOR_1501,1.0,1.0,0.0,Inpatient,357.15,0.000000
892,TEST,COLLECTOR_860,72.0,48.0,23.0,Inpatient,357.15,0.479167
893,TEST,COLLECTOR_3200,30.0,3.0,1.0,Inpatient,357.15,0.333333


In [17]:
test = test.drop(['SPLIT', 'SPECIMEN_COUNT', 'COLLECTOR_ID'], axis = 1)
test = test.loc[(test['ENCOUNTER_TYPE'] == "Emergency") | (test['ENCOUNTER_TYPE'] == "Inpatient")]
test = test.drop('ENCOUNTER_TYPE', axis=1)
test

,median_hi,REDRAW_COUNT,REDRAW_COST,REDRAW_PERCENT
0,32.0,4.0,600.00,0.235294
1,50.0,3.0,357.15,0.333333
3,9.0,0.0,357.15,0.000000
4,31.0,2.0,600.00,0.111111
6,16.0,0.0,357.15,0.000000
...,...,...,...,...
888,12.5,1.0,357.15,0.019231
890,14.0,1.0,357.15,0.032258
891,1.0,0.0,357.15,0.000000
892,72.0,23.0,357.15,0.479167


In [18]:
dtestset = xgb.DMatrix(test)
predictions = bst.predict(dtestset)
test["pred"]=predictions
test

,median_hi,REDRAW_COUNT,REDRAW_COST,REDRAW_PERCENT,pred
0,32.0,4.0,600.00,0.235294,1.953370
1,50.0,3.0,357.15,0.333333,2.510889
3,9.0,0.0,357.15,0.000000,0.304316
4,31.0,2.0,600.00,0.111111,1.311873
6,16.0,0.0,357.15,0.000000,0.332751
...,...,...,...,...,...
888,12.5,1.0,357.15,0.019231,1.501695
890,14.0,1.0,357.15,0.032258,1.442110
891,1.0,0.0,357.15,0.000000,0.356631
892,72.0,23.0,357.15,0.479167,0.846387


In [19]:
sub=pd.read_csv('/kaggle/input/aacc-2023-helpwithhemolysis/sample_solution.csv')
sub = sub.merge(test['pred'], how='left', left_index=True, right_index=True)
sub

,SPLIT,COLLECTOR_ID,pred
0,TEST,COLLECTOR_2011,1.953370
1,TEST,COLLECTOR_1550,2.510889
2,TEST,COLLECTOR_4570,NaN
3,TEST,COLLECTOR_3531,0.304316
4,TEST,COLLECTOR_5020,1.311873
...,...,...,...
890,TEST,COLLECTOR_2848,1.442110
891,TEST,COLLECTOR_1501,0.356631
892,TEST,COLLECTOR_860,0.846387
893,TEST,COLLECTOR_3200,1.144415


In [20]:
sub=sub.sort_values(by="pred",ascending=False)
sub

,SPLIT,COLLECTOR_ID,pred
183,TEST,COLLECTOR_5162,36.953739
163,TEST,COLLECTOR_2973,36.892879
850,TEST,COLLECTOR_2028,13.891359
76,TEST,COLLECTOR_1491,11.083419
786,TEST,COLLECTOR_1460,6.416948
...,...,...,...
882,TEST,COLLECTOR_3097,NaN
883,TEST,COLLECTOR_310,NaN
884,TEST,COLLECTOR_5283,NaN
889,TEST,COLLECTOR_418,NaN


In [21]:
sub[['SPLIT','COLLECTOR_ID']].to_csv('mlsub5.csv',index=False)